<a href="https://colab.research.google.com/github/TomPhanAnh/Kaggle-Solution/blob/main/Full_model_RIO_Technology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [250]:
import pandas as pd
import ast
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import datetime
from datetime import datetime
import ast # return string
# ------------------------------------------------


# ------------------------------------------------
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score


pd.set_option("display.max_columns", 50)

# **1.Input dataset** 

Lấy dữ liệu từ DYNO

## **Dữ liệu DYNO**

In [251]:
import requests
import json
import pandas as pd
import numpy as np
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pickle


class DYNO_get_data:
    
    '''Class to get data'''
    
    def __init__(self):
        print('Change directory to the folder containing data')
        print('Change name of columns in dataframe (phone and reg_citizen) for merge')
        print('If customers register ids, remember to use reg_citizen instead of citizen - which use for id from phone')
        
        self.token = "599450a72c558c1f60cf4f6c8fe5247683524c38a1afe3d348383830"

        self.API_links = ['https://gateway.dyno.me/dase/mapp/facebook/phone/',
                         'https://gateway.dyno.me/dase/mapp/citizen/phone/',
                         #'https://gateway.dyno.me/dase/demo/is_valid/facebook/',
                         #'https://gateway.dyno.me/dase/demo/name/facebook/',
                         #'https://gateway.dyno.me/dase/demo/relationship/facebook/',
                         #'https://gateway.dyno.me/dase/demo/location/facebook/',
                         #'https://gateway.dyno.me/dase/demo/location/phone/',
                         #'https://gateway.dyno.me/dase/demo/location/citizen/',
                         #'https://gateway.dyno.me/dase/demo/education/facebook/',
                         #'https://gateway.dyno.me/dase/demo/education/phone/',
                         #'https://gateway.dyno.me/dase/demo/age/facebook/',
                         #'https://gateway.dyno.me/dase/demo/age/phone/',
                         #'https://gateway.dyno.me/dase/demo/work/facebook/',
                         #'https://gateway.dyno.me/dase/demo/work/phone/',
                         #'https://gateway.dyno.me/dase/demo/face/facebook/',
                         #'https://gateway.dyno.me/dase/conn/friend_count/facebook/',
                         #'https://gateway.dyno.me/dase/conn/top_friends/facebook/',
                         #'https://gateway.dyno.me/dase/adva/arpu/facebook/',
                         #'https://gateway.dyno.me/dase/adva/arpu/phone/',
                         #'https://gateway.dyno.me/dase/adva/pay_first/facebook/',
                         #'https://gateway.dyno.me/dase/adva/pay_first/phone/',
                         #'https://gateway.dyno.me/dase/adva/credit_score/facebook/',
                         'https://gateway.dyno.me/dase/adva/credit_score/phone/',
                         #'https://gateway.dyno.me/dase/adva/post/facebook/',
                         #'https://gateway.dyno.me/dase/adva/post/phone/',
                         #'https://gateway.dyno.me/dase/adva/img_tagged/facebook/',
                         #'https://gateway.dyno.me/dase/adva/img_tagged/phone/',
                         #'https://gateway.dyno.me/dase/adva/img/facebook/',
                         #'https://gateway.dyno.me/dase/adva/img/phone/',
                         'https://gateway.dyno.me/dase/adva/ta/citizen/',
                         'https://gateway.dyno.me/dase/adva/sn/citizen/',
                         'https://gateway.dyno.me/dase/adva/fi/citizen/',
                         'https://gateway.dyno.me/dase/adva/dsi/citizen/'] 
        
        # Create dictionary of mapp to link
        self.dict_mapp_link = self.create_dict_mapp_link()
        
    def create_dict_mapp_link(self):
        dict_mapp_link = {}
        for link in self.API_links:
            list_words = link.split('/')
            mapp_name = list_words[-2] + '_to_' + list_words[-3]
            dict_mapp_link[mapp_name] = link
        
        # To map registered ids
        dict_mapp_link['reg_citizen_to_ta'] = dict_mapp_link['citizen_to_ta']
        dict_mapp_link['reg_citizen_to_sn'] = dict_mapp_link['citizen_to_sn']
        dict_mapp_link['reg_citizen_to_fi'] = dict_mapp_link['citizen_to_fi']
        dict_mapp_link['reg_citizen_to_dsi'] = dict_mapp_link['citizen_to_dsi']
        
        return dict_mapp_link
    
    def mapp_info(self, mapp, info, pbar):

        ''' Return a dict of all the queried information from kind of mapp and input info'''

        # Update progress bar
        pbar.update(1)

        # Get link API to request
        try:
            url = self.dict_mapp_link[mapp]
            r = requests.get(url + str(info), headers={'Authorization': self.token})
            content = json.loads(r.content)

            if not content['has_error']:
                return {info: content['data']}

            # Quota exceeded
            if content['error_message'] not in ['PROFILE NOT FOUND', 'PROFILE NOT CRAWLED']:
                print(content)

            return {info: np.nan}

        except Exception as e:
            print('Error at', info, 'with error:', e, '!!!')    
            
            return {}
    
    def get_data(self, list_info, mapp, max_workers=5):
    
        '''Get data for one type of mapp'''
        
        filename = mapp + '.pkl'
        threads = []
        mapp_file = {}

        # Continue to append data
        if os.path.exists(filename):
            print('File exists')
            mapp_file = pd.read_pickle(filename)

        print("Getting data with", max_workers, "workers for", filename, "!")

        with tqdm(total=len(list_info)) as pbar:
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                for info in list_info:
                    threads.append(executor.submit(self.mapp_info, mapp, info, pbar))
                for task in as_completed(threads):
                    mapp_file.update(task.result())

                    # Save file every 100 records
                    if len(mapp_file) % 100 == 0:
                        pickle.dump(mapp_file, open(filename, "wb"))
                        
        # Save file
        pickle.dump(mapp_file, open(filename, "wb"))
    
    def extract_dict_phone2id(self, filename='phone_to_citizen.pkl'):

        ''' Extract CMNDs from citizen mapping from phone '''

        phone_to_citizen = pd.read_pickle(filename)
        dict_phone2id = {}

        for k, v in phone_to_citizen.items():
            if pd.notnull(v) and 'id' in v['ids'][0]:
                dict_phone2id[k] = v['ids'][0]['id']

        return dict_phone2id

    def merge(self, df, filenames, use_id_from_phone=False):

        '''Merge all dicts to original dataframe'''

        if use_id_from_phone:
            dict_phone2id = self.extract_dict_phone2id()
            df['citizen'] = df['phone'].apply(lambda x: dict_phone2id[x] if x in dict_phone2id else np.nan)

        for filename in filenames:
            dict_mapping = pd.read_pickle(filename)
            col_name = filename.split('.')[0]
            info_name = filename.split('_to_')[0]
            df[col_name] = df[info_name].apply(lambda x: dict_mapping[x] if x in dict_mapping else np.nan)

        return df
    
    def check_error_info(self, list_info, file_name):
        
        '''Check error phones or ids while querying'''
        
        dict_mapping = pd.read_pickle(file_name)
        
        return [info for info in list_info if info not in dict_mapping]

## **1.1 Dữ liệu phân tích**

In [252]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [253]:

cd /content/drive/MyDrive/1. Colab Credit.vn/2. dataset/1. DYNO Dataset

/content/drive/MyDrive/1. Colab Credit.vn/2. dataset/1. DYNO Dataset


In [254]:
# Read file csv
df = pd.read_csv("/content/drive/MyDrive/1. Colab Credit.vn/1. FI/2. Mcredit/selected_data.csv")


# Function to calculate age
def to_datetime(x):
  try:
    return datetime.now().year - x
  except:
    return datetime.now().year - int(x[-4:])
  return np.nan

df['status'] = df['status'].astype(int)
df = df[df['status'] != 0]
df['status'] = df['status'].apply(lambda x: 0 if x == -1 else x)
df['phone'] = df['phone'].apply(lambda x: '84' + str(x))

def convert_income(x):
    try:
        x = float(x.replace('.', '').replace('$', ''))
        if x <= 100:
            return x * 1000000
        if 100 < x < 100000:
            return x * 1000
    except:
        return np.nan
df['income'] = df['income'].apply(convert_income)

df = df[['name', 'phone', 'reg_citizen', 'city', 'dob', 'income', 'status',
       'citizen', 'phone_to_citizen', 'phone_to_credit_score', 'citizen_to_fi',
       'citizen_to_sn', 'citizen_to_ta', 'citizen_to_dsi',
       'reg_citizen_to_dsi', 'reg_citizen_to_fi', 'reg_citizen_to_phone',
       'reg_citizen_to_sn', 'reg_citizen_to_ta']]
province_mcredit = ['an giang',  'ba ria - vung tau', 'bac lieu', 'bac giang', 'bac ninh',
 'ben tre', 'binh duong', 'binh dinh', 'binh phuoc', 'binh thuan', 'ca mau', 'can tho', 'da nang',
 'dak lak', 'dong nai', 'dong thap', 'gia lai', 'ha nam', 'ha noi', 'ha tinh', 'hai duong', 'hai phong',
 'hau giang', 'hung yen', 'khanh hoa', 'kien giang', 'lam dong', 'long an', 'nam dinh', 'nghe an',
 'ninh binh', 'phu tho', 'quang ninh', 'soc trang', 'tay ninh', 'thai binh', 'thai nguyen', 'thanh hoa',
 'thua thien hue', 'hue', 'tien giang', 'ho chi minh', 'tp hcm', 'vinh long', 'vinh phuc', 'yen bai']

## **1.2. Feature Engineering**

In [255]:
# remove accent function
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
def remove_accents(input_str):
  s = ''
  for c in str(input_str):
    if c in s1:
      s += s0[s1.index(c)]
    else:
      s += c
  return s

# -------------------------------------------
def transform_data(df, text, info, columns):
  columns_first = ['name', 'phone', 'reg_citizen', 'city', 'dob', 'income', 'status']
  try:
    if text != "phone_to_credit_score":
      return ast.literal_eval(df[text])[info][0][columns]
    else:
      
      return ast.literal_eval(df[text])[columns]
  except:
    return np.nan
# ------------------------------------------ Combine 2 data DYNO and customer provide
cus_loan_id = [str(int(i)) for i in df["reg_citizen"] if pd.notna(i)]
columns = ['siCode', 'address', 'dob', 'gender', 'houseHoldCode', 'maBHYT', 'maSo',
       'name', 'province', 'personalId']  
def transform_fi(df,text,text2, info,cus_loan_id, columns_reg):
  if pd.isnull(df[text]):
    text, text2 = text2, text
  try:
    df1 = pd.DataFrame(ast.literal_eval(df[text])[info], columns = columns)
    if not df[text2]:
      df1 = pd.concat([df1, pd.DataFrame(ast.literal_eval(df[text2])[info], columns = columns)]) 
    df1.drop_duplicates(inplace=  True)
    df1["province"] = df1["province"].apply(lambda x: x.replace("Tỉnh ","").replace("Thành phố ",""))
    df1["province_score"] = df1["province"].apply(lambda x: 1 if remove_accents(x).lower() in province_mcredit else 0)
    df1["fi_loan"] = df1["personalId"].apply(lambda x: 1 if x in cus_loan_id else 0) # Does people in that family also in loan
    return df1.groupby("address").agg({"houseHoldCode":"nunique", # number of household
                                           "maBHYT":"nunique", # have BHYT
                                           "maSo":"nunique", # have BHYT # flaon
                                           "province_score":"sum", # count ablility to retrive money
                                       "personalId":"nunique", # count number of people
                                           "fi_loan":"sum"}).reset_index()[columns_reg][0] # number of loan human in family
  except:
    return np.nan  
# ----------------------------------------
def job_description(x):
  if pd.isnull(x) or len(str(x)) == 0: return -1
  if not x: x = x.lower()
  level_2 = ["phó hiệu trưởng","bí thư","đoàn"," giao dịch viên", "công an", "trưởng phòng","phó phòng", "giám đốc", "quản lý","tổ trưởng", "tổ phó","hiệu trưởng","cán bộ","cb","c.b","c,b","tổ","cỏn bộ","phó","loeader",
             "trưởng","phó","tp. marketing","quản lý","chủ tịch","operator","tp","pp"]
  level_1 = ["thao  tác viên","bác sĩ","đại diện bán hàng","ks","giảng viên","văn thư","phóng viên",
             "td viên","ltv","bếp chính","tín dụng", "gv","thư ký","văn phòng","thủ kho",
             "lập trình viên","nghiên cứu viên","phiên dịch","kỹ thuật","kỹ sư","kĩ sư",
             "trợ lý","kiểm toán","kế toán","ksts","giao dịch viên", "ktv","?tv","y sĩ","y sỹ",
             "thông dịch","giáo viên","trình dược viên","văn hóa - thể thao","vp","tiếp thị",
             "giám sát","máy móc","vh","vận hành"
             "quản lí chất lượng","tiếp viên",'tiếp tân',"trợ thủ","nh©n viªn ", "bác sĩ","giám sát","thiết bị",
             "chất lượng","dược tá","kiểm hóa","y tế","nh©n viªn","trợ thủ","nhân viên","nv","n,v","thợ","bảo trì","viên"]
  if any([i for i in level_2 if i in x.lower()]):
    return 2
  elif any([i for i in level_1 if i in x.lower()]):
    return 1
  else:
    return 0
# ---------------------------------------------
def transform_another(df,text,text2, info,columns_ano, col):
  if pd.isnull(df[text]):
    text, text2 = text2, text
  try:
    if text not in ["reg_citizen_to_dsi", "citizen_to_dsi"]:
      df1 = pd.DataFrame(ast.literal_eval(df[text])[info], columns = columns_ano, index = [0])
      if not df[text2]:
        df1 = pd.concat([df1, pd.DataFrame(ast.literal_eval(df[text2])[info], columns = columns_ano)]).drop_duplicates() 
    if not df[text2]:
      df1 = pd.concat([df1, pd.DataFrame(ast.literal_eval(df[text2])[info], columns = columns_ano)]).drop_duplicates()
    df1 = df1.sort_values(by = "end_date", ascending = False).loc[0].to_frame().T
    return df1[col][0]
  except:
    return np.nan  
# -------------------------------------------------
def verify_city(x,y):
  s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
  s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
 # ----------------------------- 
  try:
    if not x or x.isnumeric(): 
      return 0
    x = remove_accents(x).title()
    y = remove_accents(y).title()
    if any([x==y, x in y, y in x, x == y.rstrip(), x in y.rstrip(), x == y.rstrip()]):
      return 1
    return 0
  except:
    return 0
# ------------------------------------------------------------------------------
def verify_citizen(x,y):
  try:
    return 1 if int(x) == int(y) else 0
  except:
    return 0

# **2. Dữ liệu đầu vào**

In [ ]:
columns_p2c = ['name', 'id', 'phone', 'address', 'ward', 'district', 'province', 'dob', 'gender']
columns_p2cc = ['action_avg', 'comment_avg', 'diff_day', 'education_level', 'freq_post','friend_education_level_0', 'friend_education_level_1', 'friend_education_level_2','friend_education_level_3', 
                'friend_pay_first', 'friend_pay_later', 'likes_count','mode_time_frame', 'post_count', 'reaction_avg', 'credit_score']
columns_fi = ["houseHoldCode","maBHYT","maSo","province_score","fi_loan"]                
columns_dsi = ['name', 'personal_id', 'gender', 'birthday', 'phone', 'permanent_residence','contact_address', 'start_date', 'end_date', 'position', 'salary', 'company_name', 
               'company_registration', 'company_address','company_tax_code', 'company_size_code', 'company_si_id', 'salary_percentile', 'si_contact_person_name', 'si_contact_person_phone',
               'si_contact_person_email','health_insurance_id', 'social_insurance_id', 'maTG', 'phuongAn','wage_coefficient', 'total_salary_percentile']
columns_sn = ['personalId', 'siCode', 'status', 'name', 'maBHYT', 'houseHoldCode', 'gender', 'dob', 'address', 'isHouseHolder']
columns_dsi = [ 'permanent_residence','contact_address', 'start_date', 'end_date', 'position', 'salary','company_name', 'company_registration', 'company_address','company_tax_code',
               'company_size_code', 'company_si_id', 'salary_percentile','si_contact_person_name', 'si_contact_person_phone', 'si_contact_person_email', 'health_insurance_id',
               'social_insurance_id', 'maTG', 'phuongAn','wage_coefficient', 'total_salary_percentile']
for i in columns_p2c:
  df[i + "p2c"] = df.apply(lambda x: transform_data(x, "phone_to_citizen", "ids", i), axis = 1)
for i in columns_p2cc:
  df[i + "p2cc"] = df.apply(lambda x: transform_data(x,"phone_to_credit_score","ids",i), axis = 1)
for i in columns_fi:
  df[i + "fi"] = df.apply(lambda x: transform_fi(x,"citizen_to_fi","reg_citizen_to_fi","fi",cus_loan_id, i), axis = 1) 
for i in columns_sn:
  df[i + "sn"] = df.apply(lambda x: transform_another(x, "citizen_to_sn","reg_citizen_to_sn","si", columns_sn,i), axis = 1)
for i in columns_dsi:
  df[i + "dsi"] = df.apply(lambda x: transform_another(x, "citizen_to_dsi","reg_citizen_to_dsi","detail_si", columns_dsi, i), axis = 1)

## **2.1 Thông tin cá nhân**

In [ ]:
df.loc[df["dob"] == '0/80', "dob"] = 2022
df.loc[df["dob"] == '8', "dob"] = 2022
df.loc[df["dob"] == '/987', "dob"] = 1987
df.loc[df["dob"] ==  '993 ', "dob"] = 1993
df.loc[df["dob"] ==  '994 ', "dob"] = 1994
df.loc[df["dob"] ==  '0200', "dob"] = 2000
df.loc[df["dob"] ==  '979 ', "dob"] = 1979
df.loc[df["dob"] ==  '19i9', "dob"] = 1999
df.loc[df["dob"] ==  '9997', "dob"] = 1997
df.loc[df["dob"] ==  'g 11', "dob"] = 2022
df.loc[df["dob"] ==  '19i9', "dob"] = 1999
df.loc[df["dob"] ==  '1089-01-01', "dob"] = 1989

df["full_info"] = df.apply(lambda x: len(df.columns) - x.isnull().sum(), axis = 1) # Check full info
df["ver_name"] = df.apply(lambda x: verify_city(x["name"],x["namep2c"]), axis = 1) # Check correct name
df["ver_id"] = df.apply(lambda x: verify_citizen(x["idp2c"], x["reg_citizen"]), axis = 1) # Ver_id information
df["dob"] = df["dob"].fillna(datetime.now())
df["dobp2c"] = df["dobp2c"].fillna(datetime.now())
df["dobp2c"] = df["dobp2c"].apply(lambda x: 2022 - int(pd.to_datetime(x).strftime("%Y")))
df["city_score"] = df["city"].apply(lambda x: 1 if x in province_mcredit else 0) # Calculate the city score
df["dob"] = df["dob"].apply(lambda x: 2022 - int(x)) # Calculate the age
df["ver_bod"] = df.apply(lambda x: verify_citizen(x["dob"],x["dobp2c"]), axis = 1) # ver_birthday
df = df.loc[(df["dob"] > 18) & (df["dob"] <= 100)] # Return last df

In [ ]:
df.drop(columns = ['phone_to_citizen', 'phone_to_credit_score', 'citizen_to_fi',
       'citizen_to_sn', 'citizen_to_ta', 'citizen_to_dsi',
       'reg_citizen_to_dsi', 'reg_citizen_to_fi', 'reg_citizen_to_phone',
       'reg_citizen_to_sn', 'reg_citizen_to_ta'], inplace = True)

## **2.2 Thông tin công việc**

In [ ]:
df["ver_BHYT"]    = df["health_insurance_iddsi"].apply(lambda x: 0 if len(str(x)) == 0 and x.isnumeric() else 1)
df["ver_BHXH"]    = df["social_insurance_iddsi"].apply(lambda x: 0 if len(str(x)) == 0 and x.isnumeric() else 1)
df["position_score"] = df["positiondsi"].apply(lambda x: job_description(x))
df["contact_addressdsi"] = df["contact_addressdsi"].apply(lambda x: x.split(",")[-1] if pd.notna(x) else x)
df["contact_addressdsi"] = df["contact_addressdsi"].apply(lambda x: x.replace(" Tỉnh ","").replace(" Thành phố ","").lower() if pd.notna(x) else x)
df["contact_addressdsi"] = df["contact_addressdsi"].apply(lambda x: 1 if remove_accents(x) in province_mcredit else 0 if pd.notna(x) else x)
df["permanent_residencedsi"] = df["permanent_residencedsi"].apply(lambda x: x.split(",")[-1] if pd.notna(x) else x )
df["permanent_residencedsi"] = df["permanent_residencedsi"].apply(lambda x: x.replace(" Tỉnh ","").replace(" Thành phố ","") if pd.notna(x) else x)
df["permanent_residencedsi"] = df["permanent_residencedsi"].apply(lambda x: 1 if remove_accents(x) in province_mcredit else 0 if pd.notna(x) else x)
df["company_size_code"] = df.apply(lambda x: x["company_size_codedsi"] if all([len(str(x["company_tax_codedsi"])) > 5, len(x["company_addressdsi"]) > 5]) else 0, axis = 1)
df["company_size_code"] = df["company_size_codedsi"].map({0:0,"A":1,"B":2,'C':3,'D':4,'E':5,'F':6,'G':7,"H":8,'I':9})
df.loc[df["salarydsi"] == "", "salarydsi"] = np.nan
df["salarydsi"] = pd.to_numeric(df['salarydsi'], errors='coerce')
df["contact_person"] = df["si_contact_person_namedsi"].apply(lambda x: 0 if len(str(x)) == 0 else 1)
df["contact_phone"] = df["si_contact_person_namedsi"].apply(lambda x: 1 if len(str(x)) != 0 else 0)
df["contact_email"] = df["si_contact_person_namedsi"].apply(lambda x: 0 if len(str(x)) == 0 and "@" in x else 1)
df["contact"] = (df["contact_email"] + df["contact_phone"] + df["contact_person"]) /3  


# **3. Training model**

In [ ]:
!pip install lightgbm==3.3.2

In [ ]:
df.select_dtypes(['int','float']).columns

In [ ]:
df.columns

In [ ]:
df["genderp2c"] = df["genderp2c"].map({"M":1, "F":0})
df["gender"] = pd.to_numeric(df.apply(lambda x: x["genderp2c"] if pd.notna(x["genderp2c"]) else x["gendersn"], axis = 1))

In [ ]:
df

In [ ]:
df[['houseHoldCodefi', 'maBHYTfi', 'maSofi', 'province_scorefi','fi_loanfi']]


In [ ]:
import lightgbm as lgbm

#     if 'credit_score' in col:
#         features.append(col)

X = df[['dob', 'ver_name', 'ver_id', 'ver_bod','ver_BHYT', 'ver_BHXH', 'city_score', 'gender', 'houseHoldCodefi', 'maBHYTfi', 'maSofi', 'province_scorefi','fi_loanfi',"personalIdfi"]]
y = df['status']
X.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold

params = {
      "objective": "binary", 
      "metric": "auc"
  }

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
models = []
feature_importance = 0
score = 0 

for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y)):
    print("Fold :", fold + 1)
    X_train, y_train = X.loc[train_idx], y[train_idx]  # Create dataset
    X_valid, y_valid = X.loc[val_idx], y[val_idx]
    lgbm_train = lgbm.Dataset(X_train, y_train) #RMSPE weight
    lgbm_valid = lgbm.Dataset(X_valid, y_valid, reference = lgbm_train)
    model = lgbm.train(params=params,   # model 
                      train_set=lgbm_train,
                      valid_sets=lgbm_valid,
                      num_boost_round=2000,
                      verbose_eval=100,
                      early_stopping_rounds=100)
    y_pred = model.predict(X_valid)# validation 
    auc = roc_auc_score(y_valid, y_pred)
    print(f'Performance of the　prediction: , AUC: {auc}')
    score += auc / 5 #keep scores and models
    models.append(model)
    feature_importance += model.feature_importance()/5
    print("*" * 100)
print(f'Final score: {score}')
feature_importance_df = pd.DataFrame(data={'columns': X.columns, 'feature_importance': feature_importance}).sort_values(by='feature_importance', ascending=False)
plt.figure(figsize=(5, 10))
sns.barplot(data=feature_importance_df, x='feature_importance', y='columns')

# **4. Test New model**

## **4.1 No Undersampling** 

In [ ]:
def overview_model(model, x_train, y_train, x_test, y_test, name):
  return pd.DataFrame({"accuracy":accuracy_score(y_test, model.predict(x_test)),
                       "recall": recall_score(y_test, model.predict(x_test)),
                       "precision":precision_score(y_test, model.predict(x_test)),
                       "f1_score":f1_score(y_test, model.predict(x_test))}, index = ["name"])

In [ ]:
import xgboost as xgb

x1 = df_input[['dob', 'ver_name', 'ver_id', 'ver_bod',
       'city_score', 'full_info', 'gender_x',
       'contact', 'houseHoldCode', 'maBHYT', 'maSo', 'personalId',
       'province_score', 'fi_loan']]
y1  = df_input["status"]

x_train,  x_test,y_train, y_test = train_test_split(x1, y1,
                                                    test_size = 0.2, 
                                                    random_state = 7, 
                                                    stratify = y1)

bst = xgb.XGBClassifier(booster = "gbtree", eta = 0.3, gamma = 0,
                    max_depth= 3,
                    sampling_method = "uniform",
                    reg_lambda = 1, 
                    alpha = 0)
bst.fit(x_train, y_train)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

knn = KNeighborsClassifier(n_neighbors = 5)
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64).reset_index()

x1 = clean_dataset(df_input[["status",'permanent_residence', 'contact_address',
       'ver_BHYT', 'ver_BHXH', 'position_score', 'total_salary_percentile',
       'workplace_score', 'company_size_code', 'houseHoldCode', 'maBHYT',
       'maSo', 'personalId', 'province_score', 'fi_loan', 'ver_name', 'ver_id',
       'ver_bod']]).drop(columns = ["index","status"])
y1  = clean_dataset(df_input[["status",'permanent_residence', 'contact_address',
       'ver_BHYT', 'ver_BHXH', 'position_score', 'total_salary_percentile',
       'workplace_score', 'company_size_code', 'houseHoldCode', 'maBHYT',
       'maSo', 'personalId', 'province_score', 'fi_loan', 'ver_name', 'ver_id',
       'ver_bod']])["status"]

x_train, x_test, y_train, y_test = train_test_split(x1, y1,
                                                    test_size = 0.2, 
                                                    random_state = 7, 
                                                    stratify = y1)

knn.fit(x_train, y_train)


In [ ]:
df